In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import scipy
from PIL import Image

In [43]:
def split_image(image3, tile_size):
    image_shape = tf.shape(image3)
    tile_rows = tf.reshape(image3, [image_shape[0], -1, tile_size[1], image_shape[2]])
    serial_tiles = tf.transpose(tile_rows, [1, 0, 2, 3])
    return tf.reshape(serial_tiles, [-1, tile_size[1], tile_size[0], image_shape[2]])

In [24]:

trdata = ImageDataGenerator()
#traindata = trdata.flow_from_directory(directory= r"C:\Users\gonza\Downloads\dataset", target_size=(400,400))
traindata = trdata.flow_from_directory(directory=r"C:\Users\47573400\Downloads\dataset", target_size=(400,400), batch_size= 1)

Found 33762 images belonging to 2 classes.


In [47]:
#16 boxes de 50x50
image = split_image(traindata.classes.take(1).shape, [50,50])
#image = tf.random.normal(shape=(1, 400, 400, 3 ))
#image = tf.shape(traindata.classes[0].image[1])
boxes = tf.random.uniform(shape=(10, 4))
box_indices = tf.random.uniform(shape=(10,)
                                , minval=0,
  maxval=1, 
  dtype=tf.int32)
tf.image.crop_and_resize(image ,boxes, box_indices, (50,50) )
#tf.image.crop_to_bounding_box(testdata.classes[0]imag, 0, 0, 2, 2)
#tf.image.crop_to_bounding_box(testdata)

InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 2 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [48]:
tf.shape(traindata.classes.take(1))

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

https://builtin.com/machine-learning/vgg16

https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4